## Introduction
This is a getting started tutorial for Gammapy.

In this tutorial we will manipulate objects representing maps in Gammapy. 

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt

## Maps
The maps package contains classes to work with sky images and cubes.

In this section, we will use a simple 2D sky image and will learn how to:
- Read sky images from FITS files
- Smooth images
- Plot images
- Cutout parts from images


In [ ]:
from gammapy.maps import Map

gc_3fhl = Map.read("$GAMMAPY_DATA/fermi-3fhl-gc/fermi-3fhl-gc-counts.fits.gz")

The image is a WcsNDMap object:

In [ ]:
print(gc_3fhl)

It contains a `WcsGeom` object and data stored as a `numpy.ndarray`.

In [ ]:
print(gc_3fhl.data)

The shape of the image is 400 x 200 pixel and it is defined using a cartesian projection in galactic coordinates.

In [ ]:
print(gc_3fhl.geom)

In [ ]:
# define the position of the Galactic center and anti-center
positions = SkyCoord([0, 180], [0, 0], frame="galactic", unit="deg")
gc_3fhl.geom.contains(positions)

Regular numpy operations can be performed on the `data` attribute.

In [ ]:
total = gc_3fhl.data.sum()
print(f"Total number of counts in the image: {total:.0f}")

To show the image on the screen we can use the plot method. It basically calls plt.imshow, passing the gc_3fhl.data attribute but in addition handles axis with world coordinates using astropy.visualization.wcsaxes and defines some defaults for nicer plots (e.g. the colormap ‘afmhot’):

In [ ]:
gc_3fhl.plot(stretch="sqrt")
plt.show()

To make the structures in the image more visible we will smooth the data using a Gaussian kernel.

In [ ]:
gc_3fhl_smoothed = gc_3fhl.smooth(kernel="gauss", width=0.1 * u.deg)
gc_3fhl_smoothed.plot(stretch="linear")
plt.show()

**Exercice** (see [Visualization in Maps API tutorial](https://docs.gammapy.org/1.2/tutorials/api/maps.html#visualizing-and-plotting)) : 
- Change the figure size
- Add a colorbar
- Change colormap (e.g. gray scale) 
- Try to change label font sizes 
- Try adding grids to the plot.

The smoothed plot already looks much nicer, but still the image is rather large. As we are mostly interested in the inner part of the image, we will cut out a quadratic region of the size 9 deg x 9 deg around the GC. Therefore we use cutout to make a cutout map:

In [ ]:
# define center and size of the cutout region
center = SkyCoord(0, 0, unit="deg", frame="galactic")
gc_3fhl_cutout = gc_3fhl_smoothed.cutout(center, 9 * u.deg)
gc_3fhl_cutout.plot(stretch="sqrt")
plt.show()

### With non-spatial axis



In [ ]:
fermi_gc = Map.read("$GAMMAPY_DATA/fermi-3fhl-gc/fermi-3fhl-gc-counts-cube.fits.gz")
print(fermi_gc)

In [ ]:
fermi_gc.smooth(kernel="gauss", width=0.3*u.deg).plot_interactive()

In [ ]:
axes = fermi_gc.smooth(kernel="gauss", width=0.3*u.deg).plot_grid(ncols=2)

### Creating from a Map Geometry
The Map object couples the data (stored as a ndarray) with a Geom object. The ~Geom object can be seen as a generalization of an astropy.wcs.WCS object, providing the information on how the data maps to physical coordinate systems. In some cases e.g. when creating many maps with the same WCS geometry it can be advantageous to first create the map geometry independent of the map object it-self:

In [ ]:
from gammapy.maps import WcsGeom
wcs_geom = WcsGeom.create(binsz=0.02, width=(10, 5), skydir=(0, 0), frame="galactic", proj="CAR")

In [ ]:
print(wcs_geom.center_skydir)

**Exercice** : Create and plot an ampty all sky map in Hammer Aitoff projection with 0.5° bin size.

### Adding Non-Spatial Axes
In many analysis scenarios we would like to add extra dimension to the maps to study e.g. energy or time dependency of the data. Those non-spatial dimensions are handled with the MapAxis object. Let us first define an energy axis, with 4 bins:

In [ ]:
from gammapy.maps import MapAxis
energy_axis = MapAxis.from_energy_bounds(0.1, 100, nbin=12, unit="TeV", name="energy")
print(energy_axis)

**Exercice:** create an **energy true** axis with around 6 bins per decade between 40 GeV and 200 TeV.

*Hint : check MapAxis.from_energy_bounds docstring.*

In [ ]:
#check edges
print(energy_axis.edges)

In [ ]:
wcs_geom_3d = WcsGeom.create(binsz=0.02, width=(10, 5), skydir=(0, 0), frame="galactic", proj="CAR", axes=[energy_axis])

In [ ]:
print(wcs_geom_3d)

### Units, arithmetics

In [ ]:
area = Map.from_geom(wcs_geom_3d, data=1, unit="m2")

In [ ]:
area.quantity.to("cm2")

In [ ]:
time = Map.from_geom(wcs_geom_3d, data=1, unit="h")

In [ ]:
exposure = time * area

In [ ]:
print(exposure)

In [ ]:
exposure.quantity.to("cm2s")

**Exercice**: build a simple integrated *intensity* map of the 3FHL GC dataset from Fermi-LAT. You can use
- the integrated exposure map at $GAMMAPY_DATA/fermi-3fhl-gc/fermi-3fhl-gc-exposure.fits.gz
- a smoothed version of the integrated counts map above

*Hint: the unit is not defined in the exposure map header. It is in $cm^2\ s$.*
*Note that `smooth` perform a normalized convolution. The resulting intesinty map is therefore flux per pixel. The peak of the emission does not give the source flux.*

### RegionGeom : geometries with one spatial bin

In [ ]:
from regions import CircleSkyRegion

GC_position = SkyCoord(359.94, -0.04, unit="deg", frame="galactic")
region = CircleSkyRegion(GC_position, radius=0.3*u.deg)

In [ ]:
from gammapy.maps import RegionGeom

In [ ]:
geom = RegionGeom.create(region, axes=[energy_axis])
map1D = Map.from_geom(geom)
print(map1D)

You can then extract informations on the `RegionGeom` as for any other `Geom`.

In [ ]:
print(geom.region)
print(f"The solid angle is {geom.solid_angle()}")

Does the geom contain a specific position?

In [ ]:
other_position = SkyCoord(0.5,0, unit="deg", frame="galactic")
geom.contains(other_position)

### Extracting a RegionNDMap (1D spectrum) from a WCSNDMap (3D cube)

You can extract the information from the 3D map and combine it into a 1D map

In [ ]:
gc_3fhl.to_region_nd_map(region)

In [ ]:
fermi_gc_region = fermi_gc.to_region_nd_map(region=region)
print(fermi_gc_region)

In [ ]:
ax = fermi_gc_region.plot()

**Exercice**: build a region map containing the counts integrated between (-0.5° < GLAT < 0.5°)